In [1]:
import faiss
import numpy as np
from utils import read_jsonl, fetch_with_retry
from sentence_transformers import SentenceTransformer

In [2]:
data = read_jsonl("../autodl-tmp/data_big_3.jsonl")

In [3]:
class VectorDatabase:
    def __init__(self, dimension):
        """初始化 FAISS 索引"""
        self.dimension = dimension
        self.model = SentenceTransformer("../all-mpnet-base-v2", local_files_only=True)
        self.index = faiss.IndexFlatL2(dimension)  # 使用 L2 距离索引
        self.texts = []  # 存储文本信息

    def add_embeddings(self, embeddings, texts):
        """添加已有的嵌入向量到 FAISS 索引"""
        embeddings = np.array(embeddings, dtype=np.float32)
        self.index.add(embeddings)
        self.texts.extend(texts)
        print(f"已添加 {len(texts)} 条数据到 FAISS 索引。")

    def search(self, query_embedding, top_k=5):
        """基于嵌入向量进行相似度搜索"""
        query_embedding = np.array([query_embedding], dtype=np.float32)
        distances, indices = self.index.search(query_embedding, top_k)
        results = [(self.texts[i], distances[0][j]) for j, i in enumerate(indices[0]) if i != -1]
        return results

    def generate_prompt(self, query, retrieved_texts):
        """基于查询与检索的文本，构造提示词"""
        prompt_template = f"""
你是一名资深舆情分析专家，请从专业角度出发，基于下列用户查询与召回的相关舆情信息，综合进行事件判断，评估该事件是否可能演变为负面舆情，并说明你的分析依据与逻辑过程。

请严格按照以下结构进行回答：

1. 舆情背景提取：  
请提炼当前舆情材料中的核心观点，明确事件的主要内容、关键参与方（如政府机构、企业、公众群体等），并指出当前公众关注的核心议题与焦点。

2. 情绪与争议因素分析：  
分析舆情材料中的情绪倾向（正面、中性或负面），识别可能激发公众情绪或引发争议的关键因素（如道德冲突、政策执行矛盾、历史积怨、社会敏感群体等），并说明其潜在敏感性。

3. 历史相似舆情参考与比较：  
参考过往与本事件相似的舆情案例（如事件类型、传播路径、结果等），对比其演化过程，并指出相似性与可能借鉴的经验或教训。

4. 风险演变路径推测：  
推测该事件未来可能的扩散或情绪演化路径（如：媒体报道、KOL扩散、二次创作、公众群体共鸣等），并指出风险节点。

5. 专业判断结论：  
结合上述分析，明确判断该事件是否具备负面舆情演变风险，并以专业语言简要说明判断依据与理由（不得使用“信息不足”或模糊判断语言）。

用户查询输入：
{user_query}

相关舆情信息：
{retrieved_texts}
"""
        return prompt_template

In [4]:
dimension = len(data[0]["embeddings"])

vector_db = VectorDatabase(dimension)

# 添加已有的 embeddings 和对应文本
embeddings = [item["embeddings"] for item in data]
texts = [item["text"] for item in data]
vector_db.add_embeddings(embeddings, texts)

已添加 8983 条数据到 FAISS 索引。


In [5]:
query = "类似杨铭宇黄焖鸡后厨卫生问题的食品安全事件在过去有哪些案例？这些事件最终的舆情走向如何？对品牌造成了什么影响？"
query_embedding = vector_db.model.encode(query, convert_to_numpy=True)  # 假设查询语句的 embedding
results = vector_db.search(query_embedding, top_k=10)

for text, score in results:
    print(f"相关文本: {text} (相似度: {score})")

相关文本: 即使是被扔在地上的食物也可能是别人需要的，该博主遇到的房屋大概率是牧民的房子，路过时若遇到难处可以在房子内吃点用点，但这位博主将食品打包带走的行为确有不妥 (相似度: 0.378437876701355)
相关文本: 该行为涉嫌违反食品安全法，市监局已查封涉案食品，肉串末烤制及流入市场，商家已自行停业整顿 (相似度: 0.3904156982898712)
相关文本: 9月2日，甘肃省靖远县食品安全委员会办公室就此事发布情况通报：9月1日晚间，媒体播出了枸杞生产区存在违规违法生产加工问题，涉及我县种植户 (相似度: 0.4185607433319092)
相关文本: 下面就给大家带来一些专家的防脱发建议：早睡早起，生活作息规律；学会减压，学习工作劳逸结合；饮食规律，多吃健康食品；正确洗护，不要经常烫染发 (相似度: 0.42031484842300415)
相关文本: 经调查，初步排除食物中毒，疑似诺如病毒感染 (相似度: 0.4298630654811859)
相关文本: 二、发展趋势由上图趋势可以看出，全网有关“绝味食品遭证监会立案”的舆情总量最高峰出现在08月15日22时 (相似度: 0.45481348037719727)
相关文本: 两人吸食完后，便想将“笑气”带至无锡 (相似度: 0.49326831102371216)
相关文本: @中储粮集团 回应称，系因作业区使用手机等电子设备产生安全隐患，针对此前粮食销售出库过程中个别客户以点概面、影响舆论的行为，中储粮集团要求直属库加强现场管理 (相似度: 0.4980199933052063)
相关文本: 夜深人静时，麻辣小龙虾、麻辣烫、烤肉、火锅、炸鸡、烧烤……各种美食萦绕脑海，也难怪网友们按捺不住食欲来吐槽 (相似度: 0.5232873558998108)
相关文本: 因食量小未能吃完午餐，小萍被幼儿园老师独自留在二楼教室反思，后被误锁在隔壁房间内，半个小时后，小萍翻越窗户、护栏坠落一楼，造成面部重伤 (相似度: 0.5273752212524414)


In [6]:
retrieved_texts = "\n".join([text for text, score in results])

In [7]:
final_prompt = vector_db.generate_prompt(query, retrieved_texts)

In [8]:
response = fetch_with_retry(final_prompt)

In [9]:
print(response.choices[0].message.content)

根据提供的舆情信息，食品安全事件存在演变为负面舆情的风险，具体分析如下：

1. **类似案例及舆情走向**  
   - **甘肃靖远县枸杞加工事件**：官方通报违规生产加工问题后，舆情聚焦于生产监管漏洞，可能引发公众对当地农产品质量的担忧，短期内负面讨论上升。  
   - **市监局查封涉案食品事件**：商家因涉嫌违法被查封并停业整顿，舆情核心指向食品安全管理失责，触发消费者信任危机，但官方介入后舆情逐渐降温。  
   - **幼儿园诺如病毒感染事件**：虽排除食物中毒，但关联到食品卫生问题，易引发家长群体对校园食品安全管理的质疑，舆情呈现局部发酵态势。

2. **对品牌的影响**  
   - **短期冲击**：涉事品牌面临口碑下滑、消费者流失，如停业整顿直接影响经营。  
   - **长期风险**：若回应迟缓或整改不力，可能被贴上“安全隐患”标签，损害品牌信誉（参考中储粮因个别事件被舆论放大后的管理争议）。  

3. **负面舆情演变可能性**  
   食品安全问题天然敏感，叠加公众健康关切，极易触发情绪化传播。若涉事方未及时公开调查结果、缺乏整改诚意，或媒体持续曝光细节（如甘肃枸杞事件中加工乱象），舆情可能从单一事件升级为行业性质疑，扩大负面影响。  

**结论**：此类事件具有较高负面舆情风险，但官方快速回应、透明处理（如市监局查封案例）可有效控制舆情扩散，降低对品牌的长期损害。
